In [1]:
from Camera import Camera
import numpy as np
from Frame import Frame
from plyfile import PlyData
import numpy as np
import argparse
from io import BytesIO
import matplotlib.pyplot as plt
import Plotters
import plotly
import plotly.graph_objects as go
import scipy.io
import pandas as pd
import Utils
from GaussianSplat import GaussianSplat
import Plotters

%matplotlib qt

# load hull
path = 'G:/My Drive/Research/gs_data/mov19_2022_03_03/'
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}
# initilize objects
frames = [1407]#list(range(500,700,13))

image_name,points_in_idx = Utils.define_frames(frames,points_3d)
cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
frames = {f'{im_name}.jpg':Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_name)}
# map 3d voxels to 2d pixels
[frames[im_name].map_3d_2d(croped_image = True) for im_name in frames.keys()]
voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)


In [ ]:
import sh_utils

input_file = "G:/My Drive/Research/gs_data/mov19_2022_03_03/output/scaling_lr0.005_percent_dense0.001_densify_grad_threshold0.0002_position_lr_init1.6e-07/point_cloud/iteration_30000/point_cloud.ply"
cam = frames['P1407CAM4.jpg'].get_croped_camera()
gs = GaussianSplat(input_file)  


gs_filtered = gs.filter(gs.projection_filter(frames,gs.xyz,croped_image = True),path = gs.path)
gs_filtered.save_gs()




In [ ]:

idx_to_clamp = (gs_filtered.color[:,0] >= 0) & (gs_filtered.color[:,1] >= 0) & (gs_filtered.color[:,2] >= 0) & (gs_filtered.color[:,0] <=1) & (gs_filtered.color[:,1] <=1) & (gs_filtered.color[:,2] <=1)
sorted_gs = GaussianSplat(vertices = gs_filtered.vertices[idx_to_clamp], sh = gs_filtered.sh[idx_to_clamp])

xyz_homo  = np.column_stack((sorted_gs.xyz,np.ones((sorted_gs.xyz.shape[0],1))))
sorted_gs.calc_cov3d()
sorted_gs.calc_cov2d(cam)
ul,br = sorted_gs.get_rect(cam)


cam.getProjectionMatrix([160,160])


p_proj = np.matmul(cam.full_proj_transform,xyz_homo.T).T
p_proj = p_proj/p_proj[:,3:]
def proj_screen(pixel,s):
    return ((pixel + 1)*s-1)*0.5
pixels = proj_screen(p_proj[:,:],160)



In [34]:
pixels

array([[136.20498766,  62.48456582, 159.49999958, 159.5       ],
       [136.22219866,  63.41715087, 159.49999958, 159.5       ],
       [136.33116131,  59.02961598, 159.49999958, 159.5       ],
       ...,
       [121.0736621 ,  53.97058387, 159.49999958, 159.5       ],
       [ 38.99775671,  98.04933212, 159.49999959, 159.5       ],
       [112.90175341,  60.19335109, 159.49999958, 159.5       ]])

In [35]:
xyz_homo  = np.column_stack((sorted_gs.xyz,np.ones((sorted_gs.xyz.shape[0],1))))
pixels2 = cam.project_on_image(xyz_homo)
pixels2

array([[136.70498766,  62.98456582],
       [136.72219866,  63.91715087],
       [136.83116131,  59.52961598],
       ...,
       [121.5736621 ,  54.47058387],
       [ 39.49775671,  98.54933212],
       [113.40175341,  60.69335109]])

In [33]:
tile = np.ones((160,160,3))

for tile_x in range(2,16):
    for tile_y in range(2,16):
        tile_coords = np.array([tile_x, tile_y])

        within_bounds = (ul <= tile_coords  ) & (br >= tile_coords )
        count_within_bounds = np.sum(within_bounds, axis=1) == 2

        pix_start = (tile_coords - 1)*16
        pix_end = (tile_coords)*16

        xyz_gs = sorted_gs.xyz[count_within_bounds]
        conic = sorted_gs.conic[count_within_bounds]
        color = sorted_gs.color[count_within_bounds]
        opacity = sorted_gs.opacity[count_within_bounds]
        proj = pixels[count_within_bounds,0:2]

        xyz_homo  = np.column_stack((xyz_gs,np.ones((xyz_gs.shape[0],1))))
        depth = np.matmul(cam.world_to_cam,xyz_homo.T).T

        idx_sort = np.argsort(depth[:,2])
        xyz_gs= xyz_gs[idx_sort]
        conic= conic[idx_sort]
        color= color[idx_sort]
        opacity= opacity[idx_sort]
        proj= proj[idx_sort]

        if sum(count_within_bounds) > 0:
            for idx_x,pix_x in enumerate(range(pix_start[0],pix_end[0])):
                for idx_y,pix_y in enumerate(range(pix_start[1],pix_end[1])):
                    pixel = [pix_start[0] + idx_x,pix_start[1] + idx_y]
                    d = np.column_stack([proj[:,0] - pixel[0],proj[:,1] - pixel[1]])
                    power = -0.5 * (conic[:,0] * d[:,0] * d[:,0] + conic[:,2] * d[:,1] * d[:,1]) - conic[:,1] * d[:,0] * d[:,1]

                    alpha = np.minimum(0.99,  conic[:,3]*np.exp(power))


                    idx_to_keep = (alpha>1/255) & (power <= 0)
                    T = 1
                    clr = [0,0,0]
                    for trans,col in zip(alpha[idx_to_keep],color[idx_to_keep]): 

                        clr += col*trans*T
                        T = T*(1-trans)
                        if T < 0.0000001:
                            break
                    tile[pixel[1],pixel[0]] = clr + T*np.array([1,1,1])
plt.figure(),plt.imshow(tile[:,:,0:3])

(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x2747fe50ca0>)

In [ ]:


SH_C0 = 0.28209479177387814
SH_C1 = 0.4886025119029199
SH_C2 = [
	1.0925484305920792,
	-1.0925484305920792,
	0.31539156525252005,
	-1.0925484305920792,
	0.5462742152960396]
SH_C3 = [
	-0.5900435899266435,
	2.890611442640554,
	-0.4570457994644658,
	0.3731763325901154,
	-0.4570457994644658,
	1.445305721320277,
	-0.5900435899266435]
pos = gs_filtered.xyz
campos = cam.t
dir = (pos - campos.T)/np.linalg.norm([pos - campos.T],axis = 1)


sh = np.column_stack((gs_filtered.vertices["f_dc_0"],gs_filtered.vertices["f_dc_1"],gs_filtered.vertices["f_dc_2"]))
sh_rest = np.column_stack([gs_filtered.vertices[f"f_rest_{idx}"] for idx in range(45)])
sh = np.column_stack((sh,sh_rest))
SH_C0 = 0.28209479177387814

result = (SH_C0 * sh[:,0:3])
x = dir[:,0][np.newaxis,:].T
y = dir[:,1][np.newaxis,:].T
z = dir[:,2][np.newaxis,:].T


result =  result - SH_C1 * y * sh[:,[3,4,5]] + SH_C1 * z * sh[:,[6,7,8]] - SH_C1 * x* sh[:,[9,10,11]]

# result = result +   SH_C3[0] * y * (3.0f * xx - yy) * sh[9] +
# 					SH_C3[1] * xy * z * sh[10] +
# 					SH_C3[2] * y * (4.0f * zz - xx - yy) * sh[11] +
# 					SH_C3[3] * z * (2.0f * zz - 3.0f * xx - 3.0f * yy) * sh[12] +
# 					SH_C3[4] * x * (4.0f * zz - xx - yy) * sh[13] +
# 					SH_C3[5] * z * (xx - yy) * sh[14] +
# 					SH_C3[6] * x * (xx - 3.0f * yy) * sh[15];
# 			}

xx = x * x
yy = y * y
zz = z * z
xy,yz,xz = x * y, y * z, x * z
result = result + SH_C2[0] * xy * sh[:,[12,13,14]]
+ SH_C2[1] * yz * sh[:,[15,16,17]]
+ SH_C2[2] * (2.0 * zz - xx - yy) * sh[:,[18,19,20]] 
+ SH_C2[3] * xz * sh[:,[21,22,23]] + SH_C2[4] * (xx - yy) * sh[:,[24,25,26]]



result = result + SH_C3[0] * y * (3.0 * xx - yy) * sh[:,[27,28,29]] +SH_C3[1] * xy * z * sh[:,[30,31,32]] 
+SH_C3[2] * y * (4.0 * zz - xx - yy) * sh[:,[33,34,35]] +SH_C3[3] * z * (2.0 * zz - 3.0 * xx - 3.0 * yy)*sh[:,[36,37,38]] 
+SH_C3[4] * x * (4.0 * zz - xx - yy) * sh[:,[39,40,41]] +SH_C3[5] * z * (xx - yy) * sh[:,[42,43,44]]
+SH_C3[6] * x * (xx - 3.0 * yy) * sh[:,[45,46,47]]


# result[:, 0] += SH_C2[0] * xy * sh[:, 4]
# result[:, 1] += SH_C2[1] * yz * sh[:, 5]
# result[:, 2] += SH_C2[2] * (2.0 * zz - xx - yy) * sh[:, 6]
# result[:, 0] += SH_C2[3] * xz * sh[:, 7]
# result[:, 0] += SH_C2[4] * (xx - yy) * sh[:, 8]

# Final adjustment
result += 0.5

# Clamping the result

# result[:,0] =result[:,0]+ SH_C1 * y * sh[:,1]
# result[:,1] = result[:,1]+SH_C1 * y * sh[:,2]
# result[:,2] = result[:,2]- SH_C1 * x * sh[:,3]


# result = result + SH_C2[0] * xy * sh[:,4] + SH_C2[1] * yz * sh[:,5] + SH_C2[2] * (2.0 * zz - xx - yy) * sh[:,6] + SH_C2[3] * xz * sh[:,7] + SH_C2[4] * (xx - yy) * sh[:,8]

In [10]:
pixels

array([[ 81.37922939,  27.66733423, 159.49999972, 159.5       ],
       [ 81.49956257,  27.70099883, 159.49999972, 159.5       ],
       [ 79.12632581,  26.30757952, 159.49999972, 159.5       ],
       ...,
       [ 73.71171868,  37.39390922, 159.49999972, 159.5       ],
       [ 73.14135876, 112.9931917 , 159.49999972, 159.5       ],
       [ 75.04693679,  45.64165071, 159.49999972, 159.5       ]])

In [11]:
plt.imshow(tile[:,:,0])

In [10]:
plt.figure(),plt.imshow(frames['P1407CAM4.jpg'].croped_image,'gray')


(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x22a56dd49a0>)

In [12]:

tile = np.ones((160,160,3))


xyz_homo  = np.column_stack((sorted_gs.xyz,np.ones((sorted_gs.xyz.shape[0],1))))
depth = np.matmul(cam.world_to_cam,xyz_homo.T).T


p_proj = np.matmul(cam.full_proj_transform,xyz_homo.T).T
p_proj = p_proj/p_proj[:,3:]

def proj_screen(pixel,s):
    return ((pixel + 1)*s-1)*0.5
pixels_gaus = proj_screen(p_proj[:,:],160)

idx_sort = np.argsort(depth[:,2])


# idx_sort = np.argsort(
#         -np.exp(sorted_gs.vertices["scale_0"] + sorted_gs.vertices["scale_1"] + sorted_gs.vertices["scale_2"])
#         / (1 + np.exp(-sorted_gs.vertices["opacity"])))

conic = sorted_gs.conic[idx_sort]
proj= pixels_gaus[idx_sort,0:2]
color= sorted_gs.color[idx_sort]

for idx_x,pix_x in enumerate(range(1,160)):
    for idx_y,pix_y in enumerate(range(1,160)):
        pixel = [pix_x,pix_y]
        d = np.column_stack([proj[:,0] - pixel[0],proj[:,1] - pixel[1]])
        within_bounds = (np.abs(d) <= 50 ) 
        count_within_bounds = np.sum(within_bounds, axis=1) == 2
        color_cur = color[count_within_bounds]
        power = -0.5 * (conic[count_within_bounds,0] * d[count_within_bounds,0] * d[count_within_bounds,0] + conic[count_within_bounds,2] * d[count_within_bounds,1] * d[count_within_bounds,1]) - conic[count_within_bounds,1] * d[count_within_bounds,0] * d[count_within_bounds,1]
        alpha = np.minimum(0.99,  conic[count_within_bounds,3]*np.exp(power) )

        idx_to_keep = (alpha>=1/255) & (power <= 0)
        T = 1
        idx = 0
        clr = [0,0,0]
        for trans,col in zip(alpha[idx_to_keep],color_cur[idx_to_keep]):
           
            clr += col*trans*T
            
            if T <= 0.0001:
                break
            T = T*(1-trans)
            
        tile[pixel[1],pixel[0],:] = np.array(clr[0:3]) + T*np.array([1,1,1])
plt.figure(),plt.imshow(tile[:,:,0:3])

(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x22a54947a60>)

In [ ]:

fig = go.Figure()
Plotters.scatter3d(fig,depth,depth[:,2]/np.max(depth[:,2]),3,opa = 1)
fig.show()

array([4., 3., 3., ..., 8., 4., 8.])

In [ ]:
plt.figure(),plt.imshow(frames['P1407CAM1.jpg'].croped_image)

(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x1d234972470>)

In [ ]:
plt.figure(),plt.imshow(tile[:,:,0:3])

(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x17403b8f520>)

In [ ]:


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def plot_covariance_as_ellipse(cov, center, ax, n_std=1.0,color_ip = 'r'):
    """
    Plot an ellipse representing a covariance matrix centered at a specific location.
    cov: 2x2 covariance matrix
    center: (x, y) tuple representing the center of the ellipse
    ax: Matplotlib Axes object for plotting
    color: Color of the ellipse
    n_std: Number of standard deviations to scale the ellipse (controls spread)
    """
    # Calculate eigenvalues and eigenvectors for axis length and orientation
    eigenvalues, eigenvectors = np.linalg.eigh(cov)
    
    # Scale eigenvalues to get width and height of the ellipse
    width, height = 2 * n_std * np.sqrt(eigenvalues)
    angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))

    # Plot ellipse with derived width, height, and angle
    ellipse = Ellipse(xy=center, width=width, height=height, angle=angle, 
                       facecolor=color_ip, lw=2)
    ax.add_patch(ellipse)

# Define a few example covariance matrices
covariance_matrices = [
    np.array([[3, 1], [1, 2]]),  # Tilted, larger spread on one axis
    np.array([[2, 0], [0, 1]]),  # Axis-aligned ellipse (wide)
    np.array([[1, 0], [0, 2]]),  # Axis-aligned ellipse (tall)
]



# Plot the ellipses for each conic matrix
fig, ax = plt.subplots()
ax.set_xlim(0, 160)
ax.set_ylim(0, 160)
ax.set_aspect('equal', 'box')
ax.set_title("Conic Matrix Ellipses")
ax.imshow(frames['P1407CAM4.jpg'].croped_image)
for i, C in enumerate(range(0,11640,2)):
    if (sorted_gs.color[i,0:3] > 0 ).all():
        if (sorted_gs.color[i,0:3]<1 ).all():
            plot_covariance_as_ellipse(sorted_gs.cov2d_matrix[i,0:2,0:2], pixel_or[i,0:2], ax, color_ip=sorted_gs.color[i,0:3])

plt.grid(True)
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.show()


In [ ]:
sorted_gs.color[i,0:3]

array([0.19556984, 0.19480544, 0.20379654], dtype=float32)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def plot_conic_ellipses(conics, positions, scale=1.0, ax=None,color_ip = [1,0,0]):
    """
    Plot ellipses based on conic coefficients.

    Parameters:
    - conics: np.ndarray of shape (n, 4), where each row is [a, b, c, opacity].
    - positions: np.ndarray of shape (n, 2), where each row is [x, y] position of the ellipse center.
    - scale: float, scaling factor for ellipse sizes.
    - ax: matplotlib.axes.Axes, optional, axis to plot on. If None, a new figure and axis will be created.
    """
    if ax is None:
        fig, ax = plt.subplots()
        
    for conic, pos,color in zip(conics, positions,color_ip):
        a, b, c, opacity = conic
        
        # Calculate the eigenvalues and eigenvectors of the conic matrix
        conic_matrix = np.array([[a, b], [b, c]])
        eigvals, eigvecs = np.linalg.eigh(conic_matrix)
        
        # Width and height of the ellipse are inversely proportional to sqrt of eigenvalues
        width = scale * (1.0 / np.sqrt(eigvals[0]))
        height = scale * (1.0 / np.sqrt(eigvals[1]))
        
        # Calculate the angle of rotation in degrees
        angle = np.degrees(np.arctan2(eigvecs[1, 0], eigvecs[0, 0]))
        
        # Draw the ellipse with specified opacity
        ellipse = Ellipse(
            xy=pos, width=width, height=height, angle=angle, facecolor=color
        )
        ax.add_patch(ellipse)
        
    # Set aspect ratio, limits, and show the plot
    ax.set_aspect('equal')
    ax.autoscale()
    ax.grid(True)
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.title("Ellipses Representing Conic Sections")
    plt.show()




# Plot the ellipses for each conic matrix
fig, ax = plt.subplots()
ax.imshow(frames['P1407CAM4.jpg'].croped_image)

idx_to_keep = (sorted_gs.color[:,0] <= 1) & (sorted_gs.color[:,0] > 0) & (sorted_gs.color[:,1] <= 1) & (sorted_gs.color[:,1] > 0) & (sorted_gs.color[:,2] <= 1) & (sorted_gs.color[:,2] > 0)
plot_conic_ellipses(sorted_gs.conic[idx_to_keep,:], pixel_or[idx_to_keep,0:2],ax = ax,color_ip =sorted_gs.color[idx_to_keep,0:3] )




In [ ]:
sorted_gs.radius

array([24., 24., 25., ..., 25., 24., 25.])

In [ ]:

tile = np.zeros((160,160))

xyz_homo  = np.column_stack((sorted_gs.xyz,np.ones((sorted_gs.xyz.shape[0],1))))
depth = np.matmul(cam.world_to_cam,xyz_homo.T).T


p_proj = np.matmul(cam.full_proj_transform,xyz_homo.T).T
p_proj = p_proj/p_proj[:,3:]

def proj_screen(pixel,s):
    return ((pixel + 1)*s-1)*0.5
pixels_gaus = proj_screen(p_proj[:,:],160)
plt.figure(),plt.scatter(pixels_gaus[:,0],pixels_gaus[:,1])

idx_sort = np.argsort(depth[:,2])
conic = sorted_gs.conic[idx_sort]
proj= pixels_gaus[idx_sort,0:2]
color= sorted_gs.color[idx_sort]
        
pixel = [40,90]
d = np.column_stack([proj[:,0] - pixel[0],proj[:,1] - pixel[1]])
within_bounds = (np.abs(d) <= 16 ) 
count_within_bounds = np.sum(within_bounds, axis=1) == 2
color_cur = color[count_within_bounds]
power = -0.5 * (conic[count_within_bounds,0] * d[count_within_bounds,0] * d[count_within_bounds,0] + conic[count_within_bounds,2] * d[count_within_bounds,1] * d[count_within_bounds,1]) - conic[count_within_bounds,1] * d[count_within_bounds,0] * d[count_within_bounds,1]
alpha = np.minimum(0.99,  conic[count_within_bounds,3]*np.exp(power)*1 )

idx_to_keep = (alpha>1/255) & (power <= 0)
T = 1
idx = 0
clr = [0,0,0,0]
for trans,col in zip(alpha[idx_to_keep],color_cur[idx_to_keep]):
    print(idx) 

    
    
    clr += 255*col[0]*trans*T
    T = T*(1-trans)
    if T < 0.0001:
        break
    print(T)
    idx += 1


0
0.010000000000000009
1
0.00010000000000000018
2


In [ ]:
sum(count_within_bounds)

2491

In [ ]:
sorted_gs.conic.shape

(11638, 4)

In [ ]:

        # tile[pixel[1],pixel[0]] = clr
                    
# plt.imshow(frames['P1407CAM2.jpg'].croped_image)
plt.figure(),plt.imshow(tile)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x231ae642dd0>)

In [ ]:
conic

array([], shape=(0, 4), dtype=float64)

In [ ]:
plt.figure(),plt.imshow(tile)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x231b118d960>)

In [ ]:
tile_x = 7
tile_y = 8

tile_coords = np.array([tile_x, tile_y])
within_bounds = (ul <= tile_coords) & (br >= tile_coords)
count_within_bounds = np.sum(within_bounds, axis=1) == 2


plt.imshow(frames['P1407CAM4.jpg'].croped_image)

plt.scatter((tile_x-1)*16,(tile_y-1)*16,marker = '*',color = 'magenta',s = 150)
plt.scatter((tile_x)*16,(tile_y)*16,marker = '*',color = 'magenta',s = 150)

ul_plt = ul[count_within_bounds]
br_plt = br[count_within_bounds]

unique_ulbr = np.unique(np.column_stack((ul_plt,br_plt)),axis = 0)
for ulbr in np.column_stack((unique_ulbr)):
    plt.scatter((ulbr[[0,2]] )*16,(ulbr[[1,3]] )*16,marker = '.',s = 150)


In [ ]:
col

array([0.53040826, 0.5300338 , 0.52964795, 0.9999459 ], dtype=float32)

In [ ]:
wak = np.array([[1,1,2,3],[3,4,5,6]])
wak[0,2]

2

(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x27c8435ff70>)

In [ ]:
plt.plot(sorted_gs.conic[:,0:2])


In [ ]:
gs_filtered.cov2d

array([[ 9.43793861e-01,  1.74958204e-03,  1.30889441e+00],
       [ 7.10886345e-01, -7.44853352e-01,  1.31638762e+00],
       [ 8.48964052e-01,  8.45039987e-01,  1.99036478e+00],
       ...,
       [ 7.43460715e+00, -4.12177970e-01,  2.28562070e-02],
       [ 6.35793848e-01, -2.73070720e-01,  2.79665555e-01],
       [ 3.04209227e+00,  2.49290590e+00,  2.05271096e+00]])

In [ ]:
conic

array([[ 1.20692405e-03,  1.34218263e-03,  1.49260790e-03,
        -1.39469409e-02],
       [ 8.31474074e-04,  1.27231876e-03,  1.94689985e-03,
        -1.47373009e-02],
       [ 1.12251007e-03,  1.34737330e-03,  1.61728310e-03,
        -1.42386162e-02],
       ...,
       [ 2.54795654e-02,  8.65315849e-03,  8.25695458e-03,
        -6.07996806e-01],
       [ 1.91356994e-02,  5.38451018e-03,  6.59602262e-03,
        -8.50763703e-01],
       [ 2.40389690e-02,  6.35234529e-03,  6.50863999e-03,
        -6.29441269e-01]])

In [ ]:
power = -0.5 * (conic[:,0] * d[:,0] * d[:,0] + conic[:,2] * d[:,1] * d[:,1]) - conic[:,1] * d[:,0] * d[:,1]
plt.plot(conic)

In [ ]:
range(int(gs_filtered.grid[0]))

range(0, 10)

In [ ]:
for idx in range(gs_filtered.grid.astype(int))

In [ ]:
x 0 10 10 20 20 30


10*i 10*i+10
0 10  10  20  20  30



In [ ]:
np.unique(np.column_stack([ul,br]))

array([[ 6.,  1., 10.,  6.],
       [ 6.,  1., 10.,  7.],
       [ 6.,  1., 10.,  6.],
       ...,
       [ 5.,  1., 10.,  6.],
       [ 0.,  4.,  5.,  9.],
       [ 4.,  1., 10.,  6.]])

In [ ]:
gs_filtered.grid

[10.9375, 10.9375]

In [ ]:
cam = frames['P1407CAM1.jpg'].get_croped_camera()
xyz_homo  = np.column_stack((gs_filtered.xyz,np.ones((gs_filtered.xyz.shape[0],1))))

cam.getProjectionMatrix([160,160])

p_proj = np.matmul(cam.full_proj_transform,xyz_homo.T).T

p_proj = np.matmul(cam.full_proj_transform,xyz_homo.T).T
def proj_screen(pixel,s):
    return ((pixel + 1)*s-1)*0.5
pixel = proj_screen(p_proj[:,:],160)

In [ ]:
plt.scatter(pixel[:,0],pixel[:,1],c = pixel[:,2])

In [ ]:
sorted_gs.xyz

array([[-4.9508931e-03, -8.8950545e-03, -1.5263262e-04],
       [-4.9518617e-03, -8.8544711e-03, -1.3404200e-04],
       [-4.9569933e-03, -9.0389168e-03, -7.8461926e-05],
       ...,
       [-4.3032514e-03, -9.2633097e-03, -2.6156742e-04],
       [-7.6228229e-04, -7.4242363e-03, -1.3504102e-03],
       [-3.9518164e-03, -9.0053175e-03, -4.2192053e-04]], dtype=float32)

In [ ]:

xyz_homo  = np.column_stack((sorted_gs.xyz,np.ones((sorted_gs.xyz.shape[0],1))))
p_proj = np.matmul(cam.full_proj_transform,xyz_homo.T).T
def proj_screen(pixel,s):
    return ((pixel + 1)*s-1)*0.5
pixel = proj_screen(p_proj[:,0:2],160)

xy_up_left_corner = np.minimum(sorted_gs.grid,(np.maximum(0,pixel - sorted_gs.radius[:,np.newaxis]) / sorted_gs.block_xy).astype(int))
xy_bot_right_corner = np.minimum(sorted_gs.grid,((np.maximum(0,(pixel + sorted_gs.radius[:,np.newaxis] + sorted_gs.block_xy - 1) / sorted_gs.block_xy)))).astype(int)

pixel = cam.project_on_image(xyz_homo)

plt.scatter(xy_up_left_corner[:,0],xy_up_left_corner[:,1])
plt.scatter(xy_bot_right_corner[:,0],xy_bot_right_corner[:,1])



In [ ]:
plt.plot((xy_up_left_corner[:,1] -xy_bot_right_corner[:,1])*(xy_up_left_corner[:,0] -xy_bot_right_corner[:,0]))

In [ ]:
wak = cam.project_on_image(xyz_homo)
plt.scatter(pixel[:,0],pixel[:,1])
plt.scatter(wak[:,0],wak[:,1])

TypeError: list indices must be integers or slices, not tuple

In [ ]:
gs_filtered.radius

34.0

In [ ]:

        
        grid_x, grid_y = grid  # unpack grid dimensions

        # Calculate min and max bounds of the rectangle
        rect_min_x = min(grid_x, max(0, int((p[0] - max_radius) / BLOCK_X)))
        rect_min_y = min(grid_y, max(0, int((p[1] - max_radius) / BLOCK_Y)))
        
        rect_max_x = min(grid_x, max(0, int((p[0] + max_radius + BLOCK_X - 1) / BLOCK_X)))
        rect_max_y = min(grid_y, max(0, int((p[1] + max_radius + BLOCK_Y - 1) / BLOCK_Y)))
        
        rect_min = (rect_min_x, rect_min_y)
        rect_max = (rect_max_x, rect_max_y)

In [ ]:
# Plots


hover_text = [f"Color: {color}" for color in gs.color[:,2]]

fig = go.Figure()
Plotters.scatter3d(fig,gs.xyz,gs.color[:,2],3)
fig.update_traces(hovertext=hover_text, hoverinfo='text')

[Plotters.scatter3d(fig,frames[list(frames.keys())[cam]].voxels_with_idx,'red',3,opa = 0.2) for cam in range(4)]
fig.show()
Plotters.scatter_projections_from_gs(frames,gs, plot_image = False)

fig = go.Figure()
Plotters.scatter3d(fig,gs_filtered.xyz,gs_filtered.color[:,2],3,opa = 0.5)
fig.update_traces(hovertext=hover_text, hoverinfo='text')
fig.show()
# fig.write_html(f"I:/My Drive/Research/gs_data/mov19_2022_03_03/output/ptcloud.html")
Plotters.scatter_projections_from_gs(frames,gs_filtered, plot_image = False)
#


FileNotFoundError: [Errno 2] No such file or directory: 'I:\\My Drive\\Research\\gs_data\\mov19_2022_03_03\\output\\ptcloud.html'

In [ ]:
scale = gs.scale[0]
scale

import numpy as np
mod = 1
S = np.eye(3) * gs_filtered.scale[:,np.newaxis,:]
S

rot = gs_filtered.rot
q = (rot.T / np.linalg.norm(rot,axis =1)).T# r, x, y, z = q
R = np.column_stack(
    [1.0 - 2.0 * (q[:,2] * q[:,2] + q[:,3] * q[:,3]), 2.0 * (q[:,1]  * q[:,2] - q[:,0] * q[:,3] ), 2.0 * (q[:,1] * q[:,3]  + q[:,0] * q[:,2]),
    2.0 * (q[:,1] * q[:,2] + q[:,0] * q[:,3] ), 1.0 - 2.0 * (q[:,1] * q[:,1] + q[:,3]  * q[:,3] ), 2.0 * (q[:,2] * q[:,3]  - q[:,0] * q[:,1]),
    2.0 * (q[:,1] * q[:,3]  - q[:,0] * q[:,2]), 2.0 * (q[:,2] * q[:,3]  + q[:,0] * q[:,1]), 1.0 - 2.0 * (q[:,1] * q[:,1] + q[:,2] * q[:,2])])
R = R.reshape(R.shape[0],3,3)
M = S @ R  # Matrix multiplication
Sigma = M.transpose(0,2,1) @ M

image_width = 160
image_height = 160
image = 'P1407CAM1.jpg'
import math
def focal2fov(focal, pixels):
    return 2*math.atan(pixels/(2*focal))

fx = frames[image].K_crop[0,0]
fy = frames[image].K_crop[1,1]

tan_fovx = focal2fov(fx, image_width)
tan_fovy = focal2fov(fy, image_height)
viewmat = frames[image].world_to_cam



t = np.matmul(viewmat , np.column_stack((gs_filtered.xyz,np.ones(gs_filtered.xyz.shape[0]))).T).T
limxy = np.column_stack((np.array([1.3 * tan_fovx]*t.shape[0]), np.array([1.3 * tan_fovy]*t.shape[0])))
t[:, :2] = np.minimum(limxy, np.maximum(-limxy, t[:,0:2]/t[:,2:])) 
zero_np = np.zeros((t.shape[0],1))


J = np.column_stack((
    fx / t[:,2:], zero_np, - (fx * t[:,0:1]) / (t[:,2:] ** 2),
    zero_np, fy / t[:,2:], - (fy * t[:,1:2]) / (t[:,2:] ** 2),
    zero_np, zero_np, zero_np
))

J = J.reshape((J.shape[0],3,3))

W = np.tile(viewmat[0:3,0:3].T,(J.shape[0],1,1))
T = W @ J
cov = T.transpose(0,2,1) @ Sigma @ T

cov = np.squeeze(np.dstack((cov[:,0,0],cov[:,0,1],cov[:,1,1])))

In [ ]:
np.squeeze(np.dstack((cov[:,0,0],cov[:,0,1],cov[:,1,1])))

(16484, 3)

In [ ]:
cov.shape

(16484, 3, 3)

In [ ]:
q = (rot.T / np.linalg.norm(rot,axis =1)).T
q

array([[ 0.96445376, -0.08434534, -0.06010166, -0.24311022],
       [ 0.99281603, -0.10408249, -0.04416541, -0.03914728],
       [ 0.9234634 , -0.3006936 , -0.11951439,  0.2061917 ],
       ...,
       [ 0.6595076 ,  0.18798089, -0.72526795,  0.06082167],
       [ 0.74300337,  0.5309335 , -0.07509685,  0.40051985],
       [ 0.7351371 ,  0.14402883,  0.44993064, -0.48620126]],
      dtype=float32)

In [ ]:
cov[0]


array([[ 0.27606161, -0.3983559 , -0.00470983],
       [-0.3983559 ,  2.07604314,  0.06530628],
       [-0.00470983,  0.06530628,  0.00236078]])

In [ ]:
covs[0][0]

array([ 0.2760616 , -0.39835588,  2.07604308])

In [ ]:
bla = R[0].copy()
bla

array([[ 9.9999857e-01, -7.5298308e-06, -9.4716688e-07],
       [ 1.8136902e-06,  9.9998641e-01, -1.9317500e-05],
       [ 7.3692609e-06,  1.7881750e-05,  9.9998659e-01]], dtype=float32)

In [ ]:
covs[0]

array([array([ 0.2760616 , -0.39835588,  2.07604308]),
       array([ 5.64345323e-09,  1.76841623e-09, -6.62277013e-10,  3.47003664e-09,
               1.88305455e-10,  1.42008109e-10])                                 ,
       array([ 0.96445376, -0.08434534, -0.06010166, -0.24311022], dtype=float32)],
      dtype=object)

In [ ]:
Sigma[0]

NameError: name 'im_name' is not defined

In [ ]:
image_width = 160
image_height = 160
image = 'P1407CAM1.jpg'
import math
def focal2fov(focal, pixels):
    return 2*math.atan(pixels/(2*focal))

fx = frames[image].K_crop[0,0]
fy = frames[image].K_crop[1,1]

tan_fovx = focal2fov(fx, image_width)
tan_fovy = focal2fov(fy, image_height)

viewmat = frames[image].world_to_cam
t = np.matmul(viewmat , np.column_stack((gs_filtered.xyz,np.ones(gs_filtered.xyz.shape[0]))).T).T

limx = np.array([1.3 * tan_fovx]*t.shape[0])
limy = np.array([1.3 * tan_fovy]*t.shape[0])

txtz = t[:,0] / t[:,2]
tytz = t[:,1] / t[:,2]
t[:,0] = np.minimum(limx, np.maximum(-limx, txtz)) * t[:,2]
t[:,1] = np.minimum(limy, np.maximum(-limy, tytz)) * t[:,2]
zero_np = np.zeros((t.shape[0],1))


J = np.column_stack((
    fx / t[:,2:], zero_np, - (fx * t[:,0:1]) / (t[:,2:] ** 2),
    zero_np, fy / t[:,2:], - (fy * t[:,1:2]) / (t[:,2:] ** 2),
    zero_np, zero_np, zero_np
))

J = J.reshape((J.shape[0],3,3))

W = np.tile(viewmat[0:3,0:3].T,(J.shape[0],1,1))
T = W @ J
cov = T.transpose(0,2,1) @ Sigma @ T

# np.array([float(cov[0, 0]), float(cov[0, 1]), float(cov[1, 1])])

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (12365,3,3)->(12365,newaxis,newaxis) (16484,3,3)->(16484,newaxis,newaxis)  and requested shape (3,3)

In [ ]:
import math

def compute_cov_3d(scale, mod, rot):
    # Create scaling matrix
    S = np.eye(3) * mod * scale

    # Normalize quaternion to get a valid rotation
    q = rot / np.linalg.norm(rot)
    r, x, y, z = q

    # Compute rotation matrix from quaternion
    R = np.array([
        [1.0 - 2.0 * (y * y + z * z), 2.0 * (x * y - r * z), 2.0 * (x * z + r * y)],
        [2.0 * (x * y + r * z), 1.0 - 2.0 * (x * x + z * z), 2.0 * (y * z - r * x)],
        [2.0 * (x * z - r * y), 2.0 * (y * z + r * x), 1.0 - 2.0 * (x * x + y * y)]
    ])

    M = S @ R  # Matrix multiplication

    # Compute 3D world covariance matrix Sigma
    Sigma = M.T @ M
    cov_3d = np.zeros(6, dtype=float)
    # Covariance is symmetric; store upper right
    cov_3d[0] = Sigma[0, 0]
    cov_3d[1] = Sigma[0, 1]
    cov_3d[2] = Sigma[0, 2]
    cov_3d[3] = Sigma[1, 1]
    cov_3d[4] = Sigma[1, 2]
    cov_3d[5] = Sigma[2, 2]

    # Covariance is symmetric; store upper right
    return cov_3d,q

def compute_cov_2d(mean,fx,fy,tan_fovx,tan_fovy,cov_3d,viewmat):

    t = np.matmul(viewmat , np.append(mean,1)).T

 
    limx = 1.3 * tan_fovx
    limy = 1.3 * tan_fovy
    txtz = t[0] / t[2]
    tytz = t[1] / t[2]
    t[0] = min(limx, max(-limx, txtz)) * t[2]
    t[1] = min(limy, max(-limy, tytz)) * t[2]

    # Compute Jacobian matrix J
    J = np.array([
        [fx / t[2], 0.0, - (fx * t[0]) / (t[2] ** 2)],
        [0.0, fy / t[2], - (fy * t[1]) / (t[2] ** 2)],
        [0, 0, 0]
    ])


    W = viewmat[0:3,0:3].T
    # Compute T matrix
    T = W @ J  # Matrix multiplication
    # Construct Vk matrix (3D covariance matrix)
    Vrk = np.array([
        [cov_3d[0], cov_3d[1], cov_3d[2]],
        [cov_3d[1], cov_3d[3], cov_3d[4]],
        [cov_3d[2], cov_3d[4], cov_3d[5]]
    ])

 

    # Compute the covariance matrix
    cov = T.T @ Vrk.T @ T

    # Apply low-pass filter: ensure Gaussian is at least one pixel wide/high
    # cov[0, 0] += 0.3
    # cov[1, 1] += 0.3

    return np.array([float(cov[0, 0]), float(cov[0, 1]), float(cov[1, 1])])



def focal2fov(focal, pixels):
    return 2*math.atan(pixels/(2*focal))

def calc_det(cov):
    return (cov[0] * cov[2] - cov[1] * cov[1])

def calc_conic(cov,det):
    det_inv = 1 / det
    return  [cov[2] * det_inv, -cov[1] * det_inv, cov[0] * det_inv ]

In [ ]:

image_width = 160
image_height = 160

fx = frames[image].K_crop[0,0]
fy = frames[image].K_crop[1,1]

tan_fovx = focal2fov(fx, image_width)
tan_fovy = focal2fov(fy, image_height)
viewmat = frames[image].world_to_cam



input_file = "D:/Documents/gray_dense_grad/output2/position_lr_init_1.6e-05_scaling_lr0.01_dens_iter500_featur0.0025/point_cloud/iteration_15000/point_cloud.ply"
pos = gs_filtered.xyz
color = gs_filtered.color
scale = gs_filtered.scale
rot = gs_filtered.rot



NameError: name 'image' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def plot_conic_ellipses(conics, positions, scale=1.0, ax=None):
    """
    Plot ellipses based on conic coefficients.

    Parameters:
    - conics: np.ndarray of shape (n, 4), where each row is [a, b, c, opacity].
    - positions: np.ndarray of shape (n, 2), where each row is [x, y] position of the ellipse center.
    - scale: float, scaling factor for ellipse sizes.
    - ax: matplotlib.axes.Axes, optional, axis to plot on. If None, a new figure and axis will be created.
    """
    if ax is None:
        fig, ax = plt.subplots()
        
    for conic, pos in zip(conics, positions):
        a, b, c, opacity = conic
        
        # Calculate the eigenvalues and eigenvectors of the conic matrix
        conic_matrix = np.array([[a, b], [b, c]])
        eigvals, eigvecs = np.linalg.eigh(conic_matrix)
        
        # Width and height of the ellipse are inversely proportional to sqrt of eigenvalues
        width = scale * (1.0 / np.sqrt(eigvals[0]))
        height = scale * (1.0 / np.sqrt(eigvals[1]))
        
        # Calculate the angle of rotation in degrees
        angle = np.degrees(np.arctan2(eigvecs[1, 0], eigvecs[0, 0]))
        
        # Draw the ellipse with specified opacity
        ellipse = Ellipse(
            xy=pos, width=width, height=height, angle=angle,
            edgecolor='blue', facecolor='cyan'
        )
        ax.add_patch(ellipse)
        
    # Set aspect ratio, limits, and show the plot
    ax.set_aspect('equal')
    ax.autoscale()
    ax.grid(True)
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.title("Ellipses Representing Conic Sections")
    plt.show()




# Plot the ellipses for each conic matrix
fig, ax = plt.subplots()
ax.imshow(frames['P1407CAM4.jpg'].croped_image)
plot_conic_ellipses(sorted_gs.conic[0::100], pixel_or[0::100,0:2],ax = ax)




import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def plot_covariance_as_ellipse(cov, center, ax, color='blue', n_std=1.0):
    """
    Plot an ellipse representing a covariance matrix centered at a specific location.
    cov: 2x2 covariance matrix
    center: (x, y) tuple representing the center of the ellipse
    ax: Matplotlib Axes object for plotting
    color: Color of the ellipse
    n_std: Number of standard deviations to scale the ellipse (controls spread)
    """
    # Calculate eigenvalues and eigenvectors for axis length and orientation
    eigenvalues, eigenvectors = np.linalg.eigh(cov)
    
    # Scale eigenvalues to get width and height of the ellipse
    width, height = 2 * n_std * np.sqrt(eigenvalues)
    angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))

    # Plot ellipse with derived width, height, and angle
    ellipse = Ellipse(xy=center, width=width, height=height, angle=angle, 
                      edgecolor=color, fc='None', lw=2)
    ax.add_patch(ellipse)

# Define a few example covariance matrices
covariance_matrices = [
    np.array([[3, 1], [1, 2]]),  # Tilted, larger spread on one axis
    np.array([[2, 0], [0, 1]]),  # Axis-aligned ellipse (wide)
    np.array([[1, 0], [0, 2]]),  # Axis-aligned ellipse (tall)
]



# Plot the ellipses for each conic matrix
fig, ax = plt.subplots()
ax.set_xlim(0, 160)
ax.set_ylim(0, 160)
ax.set_aspect('equal', 'box')
ax.set_title("Conic Matrix Ellipses")

for i, C in enumerate(range(500)):
    plot_covariance_as_ellipse(sorted_gs.cov2d_matrix[i,0:2,0:2], pixel_or[i,0:2], ax, color=f'C{i}')

plt.grid(True)
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.show()


array([[ 0.48336487, -0.49025624,  0.72526355,  0.00099953],
       [-0.8753999 , -0.27616153,  0.39674907,  0.00096338],
       [ 0.00578118, -0.82667021, -0.56265704,  0.27667279]])

In [ ]:

mod = 1.0

idx = -1
mean = pos

def get_cov(scale,rot,mean,viewmat,fx,fy,tan_fovx,tan_fovy):
    cov_3d,q = compute_cov_3d(scale, 1, rot)
    cov = compute_cov_2d(mean,fx,fy,tan_fovx,tan_fovy,cov_3d,viewmat)
    return cov,cov_3d,q

# det = calc_det(cov)
# conic = calc_conic(cov,det)

# covs = np.vstack([get_cov(scale,rot,mean,viewmat,fx,fy,tan_fovx,tan_fovy) for scale,rot,mean in zip(scale,rot,mean)])

homo_pos = frames[image].add_homo_coords(pos)
proj = frames[image].project_on_image(homo_pos,croped_camera_matrix = True)


c:\Users\Roni\anaconda3\lib\site-packages\numpy\core\shape_base.py:121: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



In [ ]:
covs[0]

array([array([ 0.2760616 , -0.39835588,  2.07604308]),
       array([ 5.64345323e-09,  1.76841623e-09, -6.62277013e-10,  3.47003664e-09,
               1.88305455e-10,  1.42008109e-10])                                 ,
       array([ 0.96445376, -0.08434534, -0.06010166, -0.24311022], dtype=float32)],
      dtype=object)

In [ ]:

image = 'P1407CAM1.jpg'


pos = gs_filtered.xyz
color = gs_filtered.color
scale = gs_filtered.scale
rot = gs_filtered.rot
indices1 = (color[:,2] >= 0.1) & (color[:,2] < 0.98) & (color[:,3] < 1)
cov = gs_filtered.cov2d
indices = (np.abs(cov[:,1])>1) & (np.abs(cov[:,2])>1) & (np.abs(cov[:,0])>1) & (color[:,2] >= 0.1) & (color[:,2] < 0.98) & (color[:,3] < 1) & (np.abs(cov[:,1])<100) & (np.abs(cov[:,2])<100) & (np.abs(cov[:,0])<100)
for_ell = gs_filtered.cov2d[indices]
homo_pos = frames[image].add_homo_coords(gs_filtered.xyz)
proj = frames[image].project_on_image(homo_pos,croped_camera_matrix = True)


for_ell_proj = proj[indices]
color_ell = gs_filtered.color[indices]

In [ ]:
for_ell_proj

array([[-0.00471224, -0.00875376, -0.00031085],
       [-0.00471327, -0.00877849, -0.00027387],
       [-0.0046862 , -0.00846439, -0.00037821],
       ...,
       [-0.00077174, -0.00744362, -0.00139848],
       [-0.00078346, -0.00743715, -0.0013813 ],
       [-0.00430325, -0.00926331, -0.00026157]], dtype=float32)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def plot_gaussian_ellipse(mean, cov_2d, ax=None, n_std=1.0, **kwargs):
    """Plot a 2D Gaussian as an ellipse."""
    if ax is None:
        ax = plt.gca()

    # Covariance matrix as 2x2
    cov_matrix = np.array([[cov_2d[0], cov_2d[1]], 
                           [cov_2d[1], cov_2d[2]]])

    # Eigenvalues and eigenvectors for the orientation and size
    eigenvals, eigenvecs = np.linalg.eigh(cov_matrix)
    order = eigenvals.argsort()[::-1]
    eigenvals, eigenvecs = eigenvals[order], eigenvecs[:, order]

    # Calculate the angle of the ellipse (in degrees)
    vx, vy = eigenvecs[:, 0]
    theta = np.degrees(np.arctan2(vy, vx))

    # Ellipse parameters
    width, height = 2 * n_std * np.sqrt(eigenvals)
    
    # Create the ellipse
    ell = Ellipse(xy=mean, width=width, height=height, angle=theta, **kwargs)
    
    ax.add_patch(ell)
    return width, height,theta


# Plot the Gaussian as an ellipse
fig, ax = plt.subplots()
ax.set_xlim(0, 160)
ax.set_ylim(0,160)
# plt.scatter(proj[:,0],proj[:,1],s = 1)

# Plot the Gaussian with 1 standard deviation
b = [plot_gaussian_ellipse(for_ell_proj, for_ell, ax=ax, facecolor = color_ell[0:3], alpha=color_ell[3]) for for_ell_proj,for_ell,color_ell in zip(for_ell_proj[0:1500],for_ell[0:1500],color_ell[0:1500])]
# Show the plot
plt.grid(True)
plt.show()

C:\Users\Roni\AppData\Local\Temp\ipykernel_25800\586283624.py:24: RuntimeWarning:

invalid value encountered in sqrt



In [ ]:
for_ell[:,0]*for_ell[:,2] - for_ell[:,1]*for_ell[:,1] 

array([[ 1.31638512e+02,  5.60281618e+01,  1.20569344e+02],
       [ 1.73860418e+02, -6.57871162e+01,  7.50580432e+01],
       [ 1.04310354e+02,  3.11110422e+01,  6.51293764e+01],
       ...,
       [ 5.11134414e-01, -1.07170627e-01,  3.54871447e-01],
       [ 4.21098920e+00,  9.85440507e+00,  2.51314231e+01],
       [ 1.18874675e+00,  2.02123437e+00,  4.89711273e+00]])

In [ ]:
n_std = 1
cov_2d = for_ell
# Covariance matrix as 2x2
cov_matrix = np.array([[cov_2d[0], cov_2d[1]], 
                        [cov_2d[1], cov_2d[2]]])

# Eigenvalues and eigenvectors for the orientation and size
eigenvals, eigenvecs = np.linalg.eigh(cov_matrix)
order = eigenvals.argsort()[::-1]
eigenvals, eigenvecs = eigenvals[order], eigenvecs[:, order]

# Calculate the angle of the ellipse (in degrees)
vx, vy = eigenvecs[:, 0]
theta = np.degrees(np.arctan2(vy, vx))

# Ellipse parameters
width, height = 2 * n_std * np.sqrt(eigenvals)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
for_ell_proj

array([[ 84.19078169,  47.06180881],
       [ 76.25174261,  87.23355188],
       [ 84.18230826,  47.46902143],
       ...,
       [ 44.42871227,  63.13493924],
       [117.26919451,  99.38322307],
       [117.19633068,  99.37221166]])

In [ ]:
im_name

'P1407CAM1.jpg'

In [ ]:
plydata = PlyData.read(input_file)
im_name = list(frames.keys())[0]
color = gs.color
pos = gs.xyz

# projection
fig,axs = plt.subplots(2,2)
for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    indices =(color[:,0] < 0.2) & (color[:,1] < 1) &(color[:,2] < 1) & (color[:,3] < 1) & (color[:,0] > 0) & (color[:,1] > 0) &(color[:,2] > 0) 

    splat = pos[indices,:]
    colors = color[indices, :]  # Filtered colors (RGB or RGBA)
    colors[:,3] = colors[:,3]
    homo_voxels_with_idx = frames[image].add_homo_coords(splat[:,0:3])
    proj = frames[image].project_on_image(homo_voxels_with_idx,croped_camera_matrix = True)
    axs[cam // 2,cam % 2].imshow(frames[image].croped_image)
    axs[cam // 2,cam % 2].scatter(proj[:,0],proj[:,1],s = 10,c = colors)
    # axs[cam // 2,cam % 2].scatter(proj[:,0],proj[:,1],s = 10,c = 'red')